# Import libraries

In [4]:
import csv
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


# Read the train and test set 

In [5]:
train_dataset='.digits/train.csv'
test_dataset='.digits/test.csv'
#read csv files
train_df=pd.read_csv(train_dataset)
test_df=pd.read_csv(test_dataset)

train_no_of_values,train_no_of_features=(train_df.shape)
#train set for validation=train_no_of_values/5
test_no_of_values,test_no_of_features=(test_df.shape)

# Identify the dependent and independent variables

In [6]:
y_train_df=train_df.iloc[:,0]
x_train_df=train_df.iloc[:,1:]

# Create numpy arrays

In [7]:
list=[]
for i in range(int(4*train_no_of_values/5)):
    list.append(y_train_df[i])
y_train=np.array(list)
print(y_train.shape)

(33600,)


In [8]:
list=[]
for i in range(int(4*train_no_of_values/5),train_no_of_values):
    list.append(y_train_df[i])
y_validate=np.array(list)
print(y_validate.shape)

(8400,)


In [9]:
list=[]
for k in range(int(4*train_no_of_values/5)):
    list2=[]
    for i in range(28):
        list1=[]
        for j in range(28):
            list1.append(x_train_df.iloc[k,i * 28 + j])
        list2.append(list1)
    list.append(list2)
x_train=np.array(list)

In [10]:
list=[]
for k in range(int(4*train_no_of_values/5),train_no_of_values):
    list2=[]
    for i in range(28):
        list1=[]
        for j in range(28):
            list1.append(x_train_df.iloc[k,i * 28 + j])
        list2.append(list1)
    list.append(list2)
x_validate=np.array(list)

In [11]:
list=[]
for k in range(test_no_of_values):
    list2=[]
    for i in range(28):
        list1=[]
        for j in range(28):
            list1.append(test_df.iloc[k,i * 28 + j])
        list2.append(list1)
    list.append(list2)
x_test=np.array(list)
print(x_test.shape) 

(28000, 28, 28)


# Preprocess the data

In [12]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_validate = x_validate.reshape(x_validate.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes=None)#convert labels to categorical features i.e.0/1
y_validate = keras.utils.to_categorical(y_validate, num_classes=None)
x_train = x_train.astype('float32')
x_validate = x_validate.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_validate /= 255
x_test /= 255

In [13]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_validate shape:', y_validate.shape)

x_train shape: (33600, 28, 28, 1)
y_train shape: (33600, 10)
x_test shape: (28000, 28, 28)
y_validate shape: (8400, 10)


# Create the model

In [14]:
num_classes = 10
epochs = 10
model = Sequential()
#input shape is needed only for first layer
#activation is 'relu' because it does not need normalisation
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))#32 filters each of size3*3
model.add(MaxPooling2D(pool_size=(2, 2)))#to create MaxPooling layer
model.add(Dropout(0.25))
model.add(Flatten())#to flatten it up into neural network 
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

# Fit the model

In [15]:
model.fit(x_train, y_train,epochs=epochs,validation_data=(x_validate, y_validate))

Train on 33600 samples, validate on 8400 samples
Epoch 1/10
33600/33600 [==============================] - 79s 2ms/step - loss: 0.2912 - accuracy: 0.9095 - val_loss: 0.0957 - val_accuracy: 0.9707
Epoch 2/10
33600/33600 [==============================] - 68s 2ms/step - loss: 0.1174 - accuracy: 0.9638 - val_loss: 0.0677 - val_accuracy: 0.9781
Epoch 3/10
33600/33600 [==============================] - 67s 2ms/step - loss: 0.0857 - accuracy: 0.9748 - val_loss: 0.0593 - val_accuracy: 0.9815
Epoch 4/10
33600/33600 [==============================] - 69s 2ms/step - loss: 0.0701 - accuracy: 0.9787 - val_loss: 0.0566 - val_accuracy: 0.9810
Epoch 5/10
33600/33600 [==============================] - 67s 2ms/step - loss: 0.0619 - accuracy: 0.9807 - val_loss: 0.0535 - val_accuracy: 0.9836
Epoch 6/10
33600/33600 [==============================] - 67s 2ms/step - loss: 0.0528 - accuracy: 0.9843 - val_loss: 0.0461 - val_accuracy: 0.9855
Epoch 7/10
33600/33600 [==============================] - 66s 2ms/ste

# Find the value accuracy and loss

In [16]:
score=model.evaluate(x_validate,y_validate,batch_size=10)
print(score)

8400/8400 [==============================] - 6s 715us/step
[0.04145519782894802, 0.9873809814453125]


# Save the model

In [57]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

Saved model to disk


# Make the predictions

In [2]:
x_test_predict=x_test.copy()#make a copy of x_test
x_test_predict = x_test_predict.reshape(x_test_predict.shape[0], 28, 28, 1)#reshape it

In [55]:
jm_list=[]
for i in range(test_no_of_values):
    #print(i)
    result_new = model.predict([x_test_predict])[i]
    number=np.argmax(result_new)
   # accuracy=max(result_new)
    jm_list.append(number)

500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749


# Check manually that the results match using the image ( just for confirmation) # Optional

In [1]:
x=int(input('Enter index of image you wish to view'))
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(x_test[x-1])

# Save results in file

In [3]:
row_list = [["ImageId", "Label"]]
for i in range(test_no_of_values):
    temp_list=[]
    temp_list.append(i+1)
    temp_list.append(jm_list[i])
    row_list.append(temp_list)

In [21]:
with open('result_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

[[7.8557860e-06 1.8801052e-07 9.9995697e-01 ... 2.9500527e-06
  1.7233033e-05 5.7252496e-06]
 [9.9960214e-01 4.6873777e-07 2.7368571e-05 ... 4.5352885e-05
  1.1389503e-05 3.8236194e-06]
 [6.5600834e-06 9.6819076e-06 1.2511021e-04 ... 4.3767691e-04
  2.4098081e-03 9.7690976e-01]
 ...
 [1.2120221e-07 1.5303051e-07 1.1326852e-06 ... 3.7040775e-06
  2.0581949e-05 8.2455108e-06]
 [3.6577319e-06 9.0515528e-07 8.8659721e-07 ... 2.1594258e-03
  1.6181373e-05 9.9119264e-01]
 [1.8914950e-04 9.5052309e-07 9.9727756e-01 ... 3.7175535e-06
  1.9820977e-03 4.7314260e-04]]
